In [1]:
import os
import shutil
import sqlite3

import pandas as pd
import requests

In [3]:
db_url = "https://storage.googleapis.com/benchmarks-artifacts/travel-db/travel2.sqlite"
local_file = "../data/travel.sqlite"
backup_file = "../data/travel.backup.sqlite"

In [ ]:
# Download and save dataset
if not os.path.exists(local_file):
    response = requests.get(db_url)
    response.raise_for_status()
    with open(local_file, "wb") as f:
        f.write(response.content)
    shutil.copy(local_file, backup_file)

In [5]:
# establish a connection to an SQLite database
conn = sqlite3.connect(local_file)

# cursor object to execute SQL commands on the database
cursor = conn.cursor()

In [15]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables

,name
0,aircrafts_data
1,airports_data
2,boarding_passes
3,bookings
4,flights
5,seats
6,ticket_flights
7,tickets
8,car_rentals
9,hotels


In [16]:
tables = tables.name.to_list()

In [18]:
tdf = {}
for table in tables:
    tdf[table] = pd.read_sql(f"SELECT * FROM {table}", conn)

In [21]:
# latest actual departure time in the flights table (acts as the reference point for time adjustment)
example_time = pd.to_datetime(
    tdf["flights"]["actual_departure"].replace("\\N", pd.NaT)
).max()
example_time

Timestamp('2024-04-30 12:05:03.561731-0400', tz='UTC-04:00')

In [22]:
current_time = pd.to_datetime("now").tz_localize(example_time.tz)
current_time

Timestamp('2025-09-18 11:11:34.926655-0400', tz='UTC-04:00')

In [24]:
time_diff = current_time - example_time
time_diff

Timedelta('505 days 23:06:31.364924')

In [25]:
tdf["bookings"]["book_date"] = (
    pd.to_datetime(tdf["bookings"]["book_date"].replace("\\N", pd.NaT), utc=True)
    + time_diff
)

In [26]:
tdf["bookings"]["book_date"][0]

Timestamp('2025-08-08 00:27:34.926655+0000', tz='UTC')

In [11]:
datetime_columns = [
    "scheduled_departure",
    "scheduled_arrival",
    "actual_departure",
    "actual_arrival"
]

for column in datetime_columns:
        tdf["flights"][column] = (
            pd.to_datetime(tdf["flights"][column].replace("\\N", pd.NaT), utc=True)
            + time_diff
        )

NameError: name 'pd' is not defined

In [28]:
tdf["flights"]["scheduled_departure"][0]

Timestamp('2025-10-14 07:05:34.926655+0000', tz='UTC')

In [ ]:
for table_name, df in tdf.items():
    df.to_sql(table_name, conn, if_exists="replace", index=False)

In [3]:
import sqlite3
conn = sqlite3.connect("/home/parichehr/PycharmProjects/NLP/travel-customer-support-agent/data/travel.sqlite")
cursor = conn.cursor()

query = """
SELECT
    t.ticket_no, t.book_ref,
    f.flight_id, f.flight_no, f.departure_airport, f.arrival_airport, f.scheduled_departure, f.scheduled_arrival,
    bp.seat_no, tf.fare_conditions
FROM
    tickets t
    JOIN ticket_flights tf ON t.ticket_no = t.ticket_no
    JOIN flights f ON tf.flight_id=f.flight_id
    JOIN boarding_passes bp ON bp.ticket_no=t.ticket_no AND bp.flight_id=f.flight_id
WHERE
    t.passenger_id = ?
"""

cursor.execute(query, ("0000 000343",))
rows = cursor.fetchall()

In [6]:
rows

[('5550005435824372',
  'D5C3E2',
  1294,
  'SU0102',
  'BSL',
  'SFO',
  '2024-04-08 14:14:03.561731-04:00',
  '2024-04-08 23:04:03.561731-04:00',
  '20B',
  'Business'),
 ('5550005435824372',
  'D5C3E2',
  1294,
  'SU0102',
  'BSL',
  'SFO',
  '2024-04-08 14:14:03.561731-04:00',
  '2024-04-08 23:04:03.561731-04:00',
  '20B',
  'Business'),
 ('5550005435824372',
  'D5C3E2',
  1294,
  'SU0102',
  'BSL',
  'SFO',
  '2024-04-08 14:14:03.561731-04:00',
  '2024-04-08 23:04:03.561731-04:00',
  '20B',
  'Business'),
 ('5550005435824372',
  'D5C3E2',
  1294,
  'SU0102',
  'BSL',
  'SFO',
  '2024-04-08 14:14:03.561731-04:00',
  '2024-04-08 23:04:03.561731-04:00',
  '20B',
  'Business'),
 ('5550005435824372',
  'D5C3E2',
  1294,
  'SU0102',
  'BSL',
  'SFO',
  '2024-04-08 14:14:03.561731-04:00',
  '2024-04-08 23:04:03.561731-04:00',
  '20B',
  'Business'),
 ('5550005435824372',
  'D5C3E2',
  1294,
  'SU0102',
  'BSL',
  'SFO',
  '2024-04-08 14:14:03.561731-04:00',
  '2024-04-08 23:04:03.561731-

In [7]:
column_names = [column[0] for column in cursor.description]

(('ticket_no', None, None, None, None, None, None),
 ('book_ref', None, None, None, None, None, None),
 ('flight_id', None, None, None, None, None, None),
 ('flight_no', None, None, None, None, None, None),
 ('departure_airport', None, None, None, None, None, None),
 ('arrival_airport', None, None, None, None, None, None),
 ('scheduled_departure', None, None, None, None, None, None),
 ('scheduled_arrival', None, None, None, None, None, None),
 ('seat_no', None, None, None, None, None, None),
 ('fare_conditions', None, None, None, None, None, None))

In [10]:
column_names = [column[0] for column in cursor.description]
results = [zip(column_names, row) for row in rows]

In [19]:
for i in results:
    print(list(i))

[('flight_no', 'SU0102'), ('departure_airport', 'BSL'), ('arrival_airport', 'SFO'), ('scheduled_departure', '2024-04-08 14:14:03.561731-04:00'), ('scheduled_arrival', '2024-04-08 23:04:03.561731-04:00'), ('seat_no', '20B'), ('fare_conditions', 'Business')]
[('ticket_no', '5550005435824372'), ('book_ref', 'D5C3E2'), ('flight_id', 1294), ('flight_no', 'SU0102'), ('departure_airport', 'BSL'), ('arrival_airport', 'SFO'), ('scheduled_departure', '2024-04-08 14:14:03.561731-04:00'), ('scheduled_arrival', '2024-04-08 23:04:03.561731-04:00'), ('seat_no', '20B'), ('fare_conditions', 'Business')]
[('ticket_no', '5550005435824372'), ('book_ref', 'D5C3E2'), ('flight_id', 1294), ('flight_no', 'SU0102'), ('departure_airport', 'BSL'), ('arrival_airport', 'SFO'), ('scheduled_departure', '2024-04-08 14:14:03.561731-04:00'), ('scheduled_arrival', '2024-04-08 23:04:03.561731-04:00'), ('seat_no', '20B'), ('fare_conditions', 'Business')]
[('ticket_no', '5550005435824372'), ('book_ref', 'D5C3E2'), ('flight_